In [1]:
import sys
sys.path.append("..")

import copy
from datasets import load_dataset
import math
import random
from sklearn.metrics import classification_report
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, DataCollatorWithPadding, \
                         AutoModelForSequenceClassification, BertForSequenceClassification

from resilient_nlp.mini_roben import Clustering, ClusterRepRecoverer, ClusterRecovererWithPassthrough
from resilient_nlp.models import BertClassifier
from resilient_nlp.perturbers import ToyPerturber, WordScramblerPerturber
from lstm import ExperimentRunner

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
imdb = load_dataset('imdb')

Reusing dataset imdb (C:\Users\Jasko\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
random.seed(11)
sampled_test_set = random.choices(imdb['test'], k=200)

In [4]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
checkpoint_finetuned = "artemis13fowl/bert-base-uncased-imdb"
model_finetuned = BertForSequenceClassification.from_pretrained(checkpoint_finetuned)

In [6]:
tp = ToyPerturber()
wsp = WordScramblerPerturber(perturb_prob=0.4, weight_add=1, weight_drop=1, weight_swap=1, weight_split_word=0, weight_merge_words=0)

In [7]:
roben_clustering = Clustering.from_pickle("../vocab100000_ed1.pkl")
roben_recoverer = ClusterRecovererWithPassthrough("cache", roben_clustering)

In [8]:
random.seed(11)
sampled_test_set_perturbed1 = copy.deepcopy(sampled_test_set)

for row in sampled_test_set_perturbed1:
    row['text'] = wsp.perturb([row['text']])[0][0]

sampled_test_set_perturbed1_roben = copy.deepcopy(sampled_test_set_perturbed1)
for row in sampled_test_set_perturbed1_roben:
    row['text'] = roben_recoverer.recover(row['text'])

In [9]:
sampled_test_set_perturbed1_roben[:5]

[{'text': 'this is so behind i don\'t know we to begin.<br /><br />te looked role is a good something point. it is a she my she center that his few this in can with the oil one from the book, who was (a bt) more credible. nco, this one is invincile, infallible, indomitable, and insipid even behind the overnflated standards that this "chics with swords" extra that or months are something out of late. she is a twetny-smething tp-model, than as a much yet rch, any a later in her academic field, a kung-fu master, a natural summer and says to he each authority to naturally top are official like play fbi agehnts. she is god.<brr /><bdr />tso be it out, she is said to be say and (due to her typically high upbringing that transformed her into a "spock") with groups us she did months we she was what is "sar wras", or "amercian idol", but yet when it\'s really important she can conveniently real himself as a top negotiator and diplomlat, be she is so superior, ya know. to top it out, she is peri

In [10]:
random.seed(11)
sampled_test_set_adv1 = []

for i in range(10):
    test_item = copy.deepcopy(sampled_test_set)

    for row in test_item:
        row['text'] = wsp.perturb([row['text']])[0][0]
    sampled_test_set_adv1.append(test_item)

In [11]:
random.seed(11)
sampled_test_set_perturbed2 = copy.deepcopy(sampled_test_set)

for row in sampled_test_set_perturbed2:
    row['text'] = tp.perturb([row['text']])[0][0]

In [12]:
max_sequence_length = 128
batch_size = 32
eval_steps = 100

In [13]:
def standard_model_predict(tokenizer, model, sentences, recoverer):
    if recoverer is not None:
        sentences = [ recoverer.recover(s.lower()) for s in sentences ]
    tokenized = tokenizer(sentences, truncation=True, padding='max_length', max_length=max_sequence_length,
                          return_tensors='pt')
    preds = model(**tokenized)
    return torch.argmax(preds.logits, dim=1)

def wrap_standard_model(tokenizer, model, recoverer=None):
    return lambda sentences: standard_model_predict(tokenizer, model, sentences, recoverer)

In [14]:
def mltokenizer_model_predict(runner, model, cls_embedding, sep_embedding, pad_embedding, sentences):
    # Truncate and lower case. Truncation is for performance only
    # sentences = [ s.lower()[:1000] for s in sentences]
    # To investigate - truncation gives only a small speedup and tanks accuracy.
    # So for now turning off truncation. This is not unfair, since we limit
    # ourselves to max_sequence_length anyway
    sentences = [ s.lower() for s in sentences]
    embedding = runner.embed(sentences=sentences,
        start_token=cls_embedding, end_token=sep_embedding, pad_token=pad_embedding,
        max_tokens=max_sequence_length)
    preds = model(inputs_embeds=embedding['inputs_embeds'], attention_mask=embedding['attention_mask'])
    return torch.argmax(preds.logits, dim=1)

def wrap_mltokenizer_model(mltokenizer_prefix, tokenizer, model):
    runner = ExperimentRunner(device)
    runner.model.load("../{}.pth".format(mltokenizer_prefix), device)
    runner.char_tokenizer.load_vocab("../{}_vocab.json".format(mltokenizer_prefix))
    cf_embedding = model.base_model.embeddings.word_embeddings
    cls_token_id = tokenizer.vocab['[CLS]']
    sep_token_id = tokenizer.vocab['[SEP]']
    pad_token_id = tokenizer.vocab['[PAD]']
    cls_embedding = cf_embedding(torch.tensor([cls_token_id])).view(-1)
    sep_embedding = cf_embedding(torch.tensor([sep_token_id])).view(-1)
    pad_embedding = cf_embedding(torch.tensor([pad_token_id])).view(-1)
    
    return lambda sentences: mltokenizer_model_predict(runner, model, cls_embedding, sep_embedding,
                                                      pad_embedding, sentences)

In [15]:
def evaluate_model(model, test_set):
    num_batches = math.ceil(len(test_set) / batch_size)
    
    sentences = [ x['text'] for x in test_set ]
    labels = [ x['label'] for x in test_set ]
    pred_batches = []
    
    for i in tqdm(range(num_batches)):
        bs = i * batch_size
        be = bs + batch_size
        
        pred_batches.append(model(sentences[bs:be]))
    preds = torch.cat(pred_batches)
    
    print(classification_report(labels, preds, digits=4))    

In [16]:
def evaluate_model_adv(model, test_sets):
    labels = [ x['label'] for x in test_sets[0] ]
    adv_preds = copy.copy(labels)
    
    for idx, test_set in tqdm(enumerate(test_sets)):
        num_batches = math.ceil(len(test_set) / batch_size)
    
        sentences = [ x['text'] for x in test_set ]
        pred_batches = []
    
        for i in range(num_batches):
            bs = i * batch_size
            be = bs + batch_size
        
            pred_batches.append(model(sentences[bs:be]))
        preds = torch.cat(pred_batches)
        
        for i in range(len(adv_preds)):
            if labels[i] == 1.0 and preds[i] == 0.0:
                adv_preds[i] = 0.0
            elif labels[i] == 0.0 and preds[i] == 1.0:
                adv_preds[i] = 1.0
    
    print(classification_report(labels, adv_preds, digits=4))    

In [17]:
baseline_model = wrap_standard_model(tokenizer, model_finetuned)

In [ ]:
mltok_model1 = wrap_mltokenizer_model('model4', tokenizer, model_finetuned)
mltok_model2 = wrap_mltokenizer_model('model5', tokenizer, model_finetuned)

In [18]:
mltok_model3 = wrap_mltokenizer_model('model6', tokenizer, model_finetuned)

In [19]:
baseline_roben_model = wrap_standard_model(tokenizer, model_finetuned, roben_recoverer)

In [20]:
evaluate_model(baseline_model, sampled_test_set)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:17<00:00,  2.49s/it]

              precision    recall  f1-score   support

           0     0.9053    0.8687    0.8866        99
           1     0.8762    0.9109    0.8932       101

    accuracy                         0.8900       200
   macro avg     0.8907    0.8898    0.8899       200
weighted avg     0.8906    0.8900    0.8899       200



In [ ]:
evaluate_model(baseline_roben_model, sampled_test_set)

In [ ]:
evaluate_model(mltok_model1, sampled_test_set)

In [ ]:
evaluate_model(mltok_model2, sampled_test_set)

In [20]:
evaluate_model(mltok_model3, sampled_test_set)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:05<00:00, 17.91s/it]

              precision    recall  f1-score   support

           0     0.5000    0.1616    0.2443        99
           1     0.5060    0.8416    0.6320       101

    accuracy                         0.5050       200
   macro avg     0.5030    0.5016    0.4381       200
weighted avg     0.5030    0.5050    0.4401       200



In [ ]:
evaluate_model(baseline_model, sampled_test_set_perturbed1)

In [ ]:
evaluate_model(baseline_roben_model, sampled_test_set_perturbed1)

In [ ]:
evaluate_model(mltok_model1, sampled_test_set_perturbed1)

In [ ]:
evaluate_model(mltok_model2, sampled_test_set_perturbed1)

In [ ]:
evaluate_model(baseline_model, sampled_test_set_perturbed2)

In [ ]:
evaluate_model(mltok_model1, sampled_test_set_perturbed2)

In [ ]:
evaluate_model(mltok_model2, sampled_test_set_perturbed2)

In [ ]:
evaluate_model_adv(baseline_model, sampled_test_set_adv1)

In [ ]:
evaluate_model_adv(baseline_roben_model, sampled_test_set_adv1)

In [ ]:
evaluate_model_adv(mltok_model1, sampled_test_set_adv1)

In [ ]:
evaluate_model_adv(mltok_model2, sampled_test_set_adv1)